In [ ]:
from selenium import webdriver
from time import sleep
import random

stop_value = 2000000 
stop = False
final_data = {}
work_stack = ["""Ahuman%231111"""]

current_player_id = work_stack[0]
driver = webdriver.Chrome()

while len(work_stack) > 0:
    try:
        url = f"https://api.tracker.gg/api/v1/valorant/matches/riot/{current_player_id}/aggregated?localOffset=-480&playlist=competitive"
        driver.get(url)
        sleep(random.randint(10,20))
        data = driver.page_source
        data = dict(eval(data
                        .replace("true", "True")
                        .replace("false", "False")
                        .replace("null", "None")
                        .replace("""<html><head><meta name="color-scheme" content="light dark"><meta charset="utf-8"></head><body><pre>""","")
                        .replace("""</pre><div class="json-formatter-container"></div></body></html>""","")))

        teammates = data['data']['teammates']
        teammates_data = []

        work_stack.pop(0)
        
        

        for teammate in teammates:
            teammates_data.append({
                "name": teammate["teammate"],
                "matches" : teammate["data"]["matches"],
                "winPct" : teammate["data"]["winPct"]
            })
            if teammate["teammate"] not in work_stack and teammate["teammate"] not in final_data:
                work_stack.append(teammate["teammate"].replace("#", "%23").replace(" ", "%20"))

        final_data[current_player_id] = teammates_data
        if len(final_data) > stop_value:
            stop = True
        print(len(final_data))

        current_player_id = work_stack[0] if len(work_stack) > 0 else None
    




    


    except Exception as e:
        print(f"An error occurred: {e}")
        work_stack.pop(0)
        current_player_id = work_stack[0] if len(work_stack) > 0 else None
        continue

#save to csv
import csv
with open('valorant_teammates_data_2.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Player ID", "Teammate Name", "Matches", "Win Percentage"])
    for player_id, teammates in final_data.items():
        for teammate in teammates:
            writer.writerow([player_id, teammate["name"], teammate["matches"], teammate["winPct"]])


driver.quit()

In [33]:
import csv
with open('valorant_teammates_data_2.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Player ID", "Teammate Name", "Matches", "Win Percentage"])
    for player_id, teammates in final_data.items():
        for teammate in teammates:
            writer.writerow([player_id, teammate["name"], teammate["matches"], teammate["winPct"]])


In [41]:
#output to nodes csv

import csv
#read the csv file
nodes = set()#unique nodes

with open('valorant_teammates_data.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        nodes.add(row["Player ID"].replace("#", "%23").replace(" ", "%20"))
        nodes.add(row["Teammate Name"].replace("#", "%23").replace(" ", "%20"))
with open('valorant_teammates_data_2.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        nodes.add(row["Player ID"].replace("#", "%23").replace(" ", "%20"))
        nodes.add(row["Teammate Name"].replace("#", "%23").replace(" ", "%20"))

#save to nodes csv
with open('valorant_teammates_data_nodes.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["id","Label"])
    for id,node in enumerate(nodes):
        writer.writerow([id,node])




#edges  Source,Target,Type,Weight
#read valorant_teammates_data_nodes.csv
name_id_map = {}
with open('valorant_teammates_data_nodes.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        name_id_map[row["Label"]] = row["id"]

data = []
with open('valorant_teammates_data.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data.append([name_id_map[row["Player ID"].replace("#", "%23").replace(" ", "%20")],
                     name_id_map[row["Teammate Name"].replace("#", "%23").replace(" ", "%20")],
                     "Undirected",
                     row["Matches"]])
with open('valorant_teammates_data_2.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data.append([name_id_map[row["Player ID"].replace("#", "%23").replace(" ", "%20")],
                     name_id_map[row["Teammate Name"].replace("#", "%23").replace(" ", "%20")],
                     "Undirected",
                     row["Matches"]])
        
#save to edges csv
with open('valorant_teammates_data_edges.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Source", "Target", "Type", "Weight"])
    for edge in data:
        writer.writerow(edge)

